<a href="https://colab.research.google.com/github/Ron-levi1/Social-Media-Advertisement-Performance/blob/main/part_1_Data_Preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Importing Libraries and Loading the Dataset

In [ ]:
!pip install pandas
import pandas as pd
!pip install numpy
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
import re
import numpy as np
import pandas as pd
import string
%matplotlib inline

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("alperenmyung/social-media-advertisement-performance")

print("Path to dataset files:", path)

100%|██████████| 15.4M/15.4M [00:00<00:00, 133MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/alperenmyung/social-media-advertisement-performance/versions/1


In [ ]:
df1 = pd.read_csv(path + "/ad_events.csv")
df2 = pd.read_csv(path + "/ads.csv")
df3 = pd.read_csv(path + "/campaigns.csv")
df4 = pd.read_csv(path + "/users.csv")

In [ ]:
print(df1.head())
print(df2.head())
print(df3.head())
print(df4.head())

   event_id  ad_id user_id            timestamp day_of_week time_of_day  \
0         1    197   2359b  2025-07-26 00:19:56    Saturday       Night   
1         2     51   f9c67  2025-06-15 08:28:07      Sunday     Morning   
2         3     46   5b868  2025-06-27 00:40:02      Friday       Night   
3         4    166   3d440  2025-06-05 19:20:45    Thursday     Evening   
4         5     52   68f1a  2025-07-22 08:30:29     Tuesday     Morning   

   event_type  
0        Like  
1       Share  
2  Impression  
3  Impression  
4  Impression  
   ad_id  campaign_id ad_platform   ad_type target_gender target_age_group  \
0      1           28    Facebook     Video        Female            35-44   
1      2           33    Facebook   Stories           All            25-34   
2      3           20   Instagram  Carousel           All            25-34   
3      4           28    Facebook   Stories        Female            25-34   
4      5           24   Instagram     Image        Female      

In [ ]:
df4.user_id.value_counts() #this shows that we have duplicated event_id but its suppuse to be uniqe

,count
user_id,
699a5,2
9e923,2
5bdf4,2
7332b,2
ce3fc,2
...,...
a14c9,1
d60c8,1
db624,1


#### cleaning the duplicates in users data frame (df4):

###### 1. remove the single row where user_gender == 'Other' for each user who appears multiple times in the dataset and has exactly one occurrence of "Other" in their gender values

In [ ]:
df4_clean = df4.copy()


multi = df4_clean.groupby('user_id')['user_id'].transform('size') > 1


exact_one_other = (
    df4_clean.groupby('user_id')['user_gender']
       .transform(lambda x: (x == 'Other').sum() == 1)
)


df4_clean = df4_clean[~(multi & exact_one_other & (df4_clean['user_gender'] == 'Other'))]
df4_clean

,user_id,user_gender,user_age,age_group,country,location,interests
0,a2474,Female,24,18-24,United Kingdom,New Mariomouth,"fitness, health"
1,141e5,Male,21,18-24,Germany,Danielsfort,"food, fitness, lifestyle"
2,34db0,Male,27,25-34,Australia,Vincentchester,"fashion, news"
3,20d08,Female,28,25-34,India,Lisaport,"health, news, finance"
4,9e830,Male,28,25-34,United States,Brownmouth,"health, photography, lifestyle"
...,...,...,...,...,...,...,...
9995,24364,Male,18,18-24,United States,Curtisside,"travel, fashion, art"
9996,68b82,Male,24,18-24,Mexico,Brownland,finance
9997,39f39,Male,29,25-34,United States,Watersburgh,health
9998,0b8e7,Male,31,25-34,United Kingdom,South Kenneth,"art, fashion"


###### 2. identify users (user_id) who have both "Female" and "Male" entries in their records, and removes only the "Female" rows for those users, keeping the "Male" entries when both genders exist.

In [ ]:
has_female = df4_clean.groupby('user_id')['user_gender'] \
    .transform(lambda x: 'Female' in x.tolist())

has_male = df4_clean.groupby('user_id')['user_gender'] \
    .transform(lambda x: 'Male' in x.tolist())

drop_mask = has_female & has_male & (df4_clean['user_gender'] == 'Female')

df4_clean = df4_clean[~drop_mask]
df4_clean

,user_id,user_gender,user_age,age_group,country,location,interests
0,a2474,Female,24,18-24,United Kingdom,New Mariomouth,"fitness, health"
1,141e5,Male,21,18-24,Germany,Danielsfort,"food, fitness, lifestyle"
2,34db0,Male,27,25-34,Australia,Vincentchester,"fashion, news"
3,20d08,Female,28,25-34,India,Lisaport,"health, news, finance"
4,9e830,Male,28,25-34,United States,Brownmouth,"health, photography, lifestyle"
...,...,...,...,...,...,...,...
9995,24364,Male,18,18-24,United States,Curtisside,"travel, fashion, art"
9996,68b82,Male,24,18-24,Mexico,Brownland,finance
9997,39f39,Male,29,25-34,United States,Watersburgh,health
9998,0b8e7,Male,31,25-34,United Kingdom,South Kenneth,"art, fashion"


###### 3. remove the single row where country == 'United States' for each user who appears multiple times in the dataset and has exactly one occurrence of "United States"

In [ ]:
# find event_ids that appear more than once
multi = df4_clean.groupby('user_id')['user_id'].transform('size') > 1

# find event_ids where exactly one row has gender "other"
exact_one_other = (
    df4_clean.groupby('user_id')['country']
       .transform(lambda x: (x == 'United States').sum() == 1)
)

# drop only the 'other' row in those groups
df4_clean = df4_clean[~(multi & exact_one_other & (df4_clean['country'] == 'United States'))]
df4_clean

,user_id,user_gender,user_age,age_group,country,location,interests
0,a2474,Female,24,18-24,United Kingdom,New Mariomouth,"fitness, health"
1,141e5,Male,21,18-24,Germany,Danielsfort,"food, fitness, lifestyle"
2,34db0,Male,27,25-34,Australia,Vincentchester,"fashion, news"
3,20d08,Female,28,25-34,India,Lisaport,"health, news, finance"
4,9e830,Male,28,25-34,United States,Brownmouth,"health, photography, lifestyle"
...,...,...,...,...,...,...,...
9995,24364,Male,18,18-24,United States,Curtisside,"travel, fashion, art"
9996,68b82,Male,24,18-24,Mexico,Brownland,finance
9997,39f39,Male,29,25-34,United States,Watersburgh,health
9998,0b8e7,Male,31,25-34,United Kingdom,South Kenneth,"art, fashion"


###### 4. manually removes a few specific rows from the dataset by their index values, as part of a random data reduction process

In [ ]:
df4_clean = df4_clean.drop(index=[1294, 1739 ,5384,7706 , 8207 , 802	, 5009 , 1996 , 4896])
df4_clean

,user_id,user_gender,user_age,age_group,country,location,interests
0,a2474,Female,24,18-24,United Kingdom,New Mariomouth,"fitness, health"
1,141e5,Male,21,18-24,Germany,Danielsfort,"food, fitness, lifestyle"
2,34db0,Male,27,25-34,Australia,Vincentchester,"fashion, news"
3,20d08,Female,28,25-34,India,Lisaport,"health, news, finance"
4,9e830,Male,28,25-34,United States,Brownmouth,"health, photography, lifestyle"
...,...,...,...,...,...,...,...
9995,24364,Male,18,18-24,United States,Curtisside,"travel, fashion, art"
9996,68b82,Male,24,18-24,Mexico,Brownland,finance
9997,39f39,Male,29,25-34,United States,Watersburgh,health
9998,0b8e7,Male,31,25-34,United Kingdom,South Kenneth,"art, fashion"


###### verify that no duplicate user_id values remain in the dataset by checking the number of occurrences of each user_id

In [ ]:
df4_clean.user_id.value_counts()

,count
user_id,
99e3f,1
a2474,1
141e5,1
34db0,1
20d08,1
...,...
e37ba,1
44b7d,1
7dfcf,1


#### Uniting Tables

###### Merge df1 and df2 on the common column ad_id using an outer join to include all rows from both tables.

In [ ]:
merged_df1_2= pd.merge(df1, df2, on=['ad_id'], how='outer')
merged_df1_2

,event_id,ad_id,user_id,timestamp,day_of_week,time_of_day,event_type,campaign_id,ad_platform,ad_type,target_gender,target_age_group,target_interests
0,65,1,e82c1,2025-06-28 10:56:52,Saturday,Morning,Impression,28,Facebook,Video,Female,35-44,"art, technology"
1,228,1,73d19,2025-07-11 22:34:14,Friday,Evening,Impression,28,Facebook,Video,Female,35-44,"art, technology"
2,391,1,a7530,2025-05-21 08:41:59,Wednesday,Morning,Impression,28,Facebook,Video,Female,35-44,"art, technology"
3,491,1,ca484,2025-06-26 14:32:05,Thursday,Afternoon,Impression,28,Facebook,Video,Female,35-44,"art, technology"
4,572,1,8c871,2025-05-19 05:44:09,Monday,Night,Impression,28,Facebook,Video,Female,35-44,"art, technology"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
399995,398935,200,9bef6,2025-05-28 16:57:07,Wednesday,Afternoon,Click,44,Instagram,Carousel,Male,18-24,photography
399996,399277,200,817c9,2025-08-05 22:00:50,Tuesday,Evening,Impression,44,Instagram,Carousel,Male,18-24,photography
399997,399570,200,a5e53,2025-08-06 02:02:32,Wednesday,Night,Click,44,Instagram,Carousel,Male,18-24,photography
399998,399762,200,a0f9a,2025-07-24 09:24:13,Thursday,Morning,Impression,44,Instagram,Carousel,Male,18-24,photography


###### Merge merged_df1_2 and df3 on the common column campaign_id using an inner join to keep only the rows with matching campaign information and remove unrelated rows without additional data

In [ ]:
merged_df1_2_3= pd.merge(merged_df1_2, df3, on=['campaign_id'], how='inner')
merged_df1_2_3

,event_id,ad_id,user_id,timestamp,day_of_week,time_of_day,event_type,campaign_id,ad_platform,ad_type,target_gender,target_age_group,target_interests,name,start_date,end_date,duration_days,total_budget
0,65,1,e82c1,2025-06-28 10:56:52,Saturday,Morning,Impression,28,Facebook,Video,Female,35-44,"art, technology",Campaign_28_Winter,2025-05-09,2025-06-30,52,32844.79
1,228,1,73d19,2025-07-11 22:34:14,Friday,Evening,Impression,28,Facebook,Video,Female,35-44,"art, technology",Campaign_28_Winter,2025-05-09,2025-06-30,52,32844.79
2,391,1,a7530,2025-05-21 08:41:59,Wednesday,Morning,Impression,28,Facebook,Video,Female,35-44,"art, technology",Campaign_28_Winter,2025-05-09,2025-06-30,52,32844.79
3,491,1,ca484,2025-06-26 14:32:05,Thursday,Afternoon,Impression,28,Facebook,Video,Female,35-44,"art, technology",Campaign_28_Winter,2025-05-09,2025-06-30,52,32844.79
4,572,1,8c871,2025-05-19 05:44:09,Monday,Night,Impression,28,Facebook,Video,Female,35-44,"art, technology",Campaign_28_Winter,2025-05-09,2025-06-30,52,32844.79
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399995,398935,200,9bef6,2025-05-28 16:57:07,Wednesday,Afternoon,Click,44,Instagram,Carousel,Male,18-24,photography,Campaign_44_Q3,2025-07-14,2025-09-22,70,15914.87
399996,399277,200,817c9,2025-08-05 22:00:50,Tuesday,Evening,Impression,44,Instagram,Carousel,Male,18-24,photography,Campaign_44_Q3,2025-07-14,2025-09-22,70,15914.87
399997,399570,200,a5e53,2025-08-06 02:02:32,Wednesday,Night,Click,44,Instagram,Carousel,Male,18-24,photography,Campaign_44_Q3,2025-07-14,2025-09-22,70,15914.87
399998,399762,200,a0f9a,2025-07-24 09:24:13,Thursday,Morning,Impression,44,Instagram,Carousel,Male,18-24,photography,Campaign_44_Q3,2025-07-14,2025-09-22,70,15914.87


###### Merge `merged_df1_2_3` and `df4_clean` on the common column `user_id` to combine user-related information with the existing merged dataset.

In [ ]:
merged_df1_2_3_4= pd.merge(merged_df1_2_3, df4_clean, on=['user_id'], how='inner')
merged_df1_2_3_4

,event_id,ad_id,user_id,timestamp,day_of_week,time_of_day,event_type,campaign_id,ad_platform,ad_type,...,start_date,end_date,duration_days,total_budget,user_gender,user_age,age_group,country,location,interests
0,65,1,e82c1,2025-06-28 10:56:52,Saturday,Morning,Impression,28,Facebook,Video,...,2025-05-09,2025-06-30,52,32844.79,Male,37,35-44,United States,Joneshaven,art
1,228,1,73d19,2025-07-11 22:34:14,Friday,Evening,Impression,28,Facebook,Video,...,2025-05-09,2025-06-30,52,32844.79,Male,28,25-34,United Kingdom,Gallowaytown,"fashion, photography"
2,391,1,a7530,2025-05-21 08:41:59,Wednesday,Morning,Impression,28,Facebook,Video,...,2025-05-09,2025-06-30,52,32844.79,Female,55,55-65,Australia,Mariaview,"news, fashion, photography"
3,491,1,ca484,2025-06-26 14:32:05,Thursday,Afternoon,Impression,28,Facebook,Video,...,2025-05-09,2025-06-30,52,32844.79,Male,40,35-44,France,Blackburgh,"technology, lifestyle"
4,572,1,8c871,2025-05-19 05:44:09,Monday,Night,Impression,28,Facebook,Video,...,2025-05-09,2025-06-30,52,32844.79,Female,17,16-17,Canada,Nicolechester,fashion
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399995,398935,200,9bef6,2025-05-28 16:57:07,Wednesday,Afternoon,Click,44,Instagram,Carousel,...,2025-07-14,2025-09-22,70,15914.87,Other,35,35-44,Canada,Jamesfurt,finance
399996,399277,200,817c9,2025-08-05 22:00:50,Tuesday,Evening,Impression,44,Instagram,Carousel,...,2025-07-14,2025-09-22,70,15914.87,Female,24,18-24,Canada,Port Sara,fashion
399997,399570,200,a5e53,2025-08-06 02:02:32,Wednesday,Night,Click,44,Instagram,Carousel,...,2025-07-14,2025-09-22,70,15914.87,Female,23,18-24,Brazil,Marktown,travel
399998,399762,200,a0f9a,2025-07-24 09:24:13,Thursday,Morning,Impression,44,Instagram,Carousel,...,2025-07-14,2025-09-22,70,15914.87,Male,18,18-24,Germany,South Brianberg,"fashion, food, fitness"


##### Save the merged dataset as a CSV file named `merged_df1_2_3_4.csv` without including index numbers, and then load it back into a new DataFrame called `df`.

In [ ]:
merged_df1_2_3_4.to_csv('merged_df1_2_3_4.csv', index=False)
df = pd.read_csv('merged_df1_2_3_4.csv')

In [ ]:
df

,event_id,ad_id,user_id,timestamp,day_of_week,time_of_day,event_type,campaign_id,ad_platform,ad_type,...,start_date,end_date,duration_days,total_budget,user_gender,user_age,age_group,country,location,interests
0,65,1,e82c1,2025-06-28 10:56:52,Saturday,Morning,Impression,28,Facebook,Video,...,2025-05-09,2025-06-30,52,32844.79,Male,37,35-44,United States,Joneshaven,art
1,228,1,73d19,2025-07-11 22:34:14,Friday,Evening,Impression,28,Facebook,Video,...,2025-05-09,2025-06-30,52,32844.79,Male,28,25-34,United Kingdom,Gallowaytown,"fashion, photography"
2,391,1,a7530,2025-05-21 08:41:59,Wednesday,Morning,Impression,28,Facebook,Video,...,2025-05-09,2025-06-30,52,32844.79,Female,55,55-65,Australia,Mariaview,"news, fashion, photography"
3,491,1,ca484,2025-06-26 14:32:05,Thursday,Afternoon,Impression,28,Facebook,Video,...,2025-05-09,2025-06-30,52,32844.79,Male,40,35-44,France,Blackburgh,"technology, lifestyle"
4,572,1,8c871,2025-05-19 05:44:09,Monday,Night,Impression,28,Facebook,Video,...,2025-05-09,2025-06-30,52,32844.79,Female,17,16-17,Canada,Nicolechester,fashion
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399995,398935,200,9bef6,2025-05-28 16:57:07,Wednesday,Afternoon,Click,44,Instagram,Carousel,...,2025-07-14,2025-09-22,70,15914.87,Other,35,35-44,Canada,Jamesfurt,finance
399996,399277,200,817c9,2025-08-05 22:00:50,Tuesday,Evening,Impression,44,Instagram,Carousel,...,2025-07-14,2025-09-22,70,15914.87,Female,24,18-24,Canada,Port Sara,fashion
399997,399570,200,a5e53,2025-08-06 02:02:32,Wednesday,Night,Click,44,Instagram,Carousel,...,2025-07-14,2025-09-22,70,15914.87,Female,23,18-24,Brazil,Marktown,travel
399998,399762,200,a0f9a,2025-07-24 09:24:13,Thursday,Morning,Impression,44,Instagram,Carousel,...,2025-07-14,2025-09-22,70,15914.87,Male,18,18-24,Germany,South Brianberg,"fashion, food, fitness"


#### Converting date columns to datetime format and extracting individual time components such as year, month, day, hour, minute, and second from the timestamp column for further analysis.


In [ ]:
df["start_date"] = pd.to_datetime(df["start_date"])
df["end_date"] = pd.to_datetime(df["end_date"])

In [ ]:
df["timestamp"] = pd.to_datetime(df["timestamp"])

In [ ]:
df["year"]  = df["timestamp"].dt.year
df["month"] = df["timestamp"].dt.month
# Extracting both numeric and named month columns from the timestamp
df["month_name"] = df["timestamp"].dt.month_name()

# Ordering month names correctly for proper sorting and visualization
month_order = ["January","February","March","April","May","June","July",
               "August","September","October","November","December"]
df["month_name"] = pd.Categorical(df["month_name"], categories=month_order, ordered=True)

df["day"]   = df["timestamp"].dt.day
df["hour"]  = df["timestamp"].dt.hour
df["minute"] = df["timestamp"].dt.minute
df["second"] = df["timestamp"].dt.second

#### Adding a new column that identifies whether the day of the week falls between Saturday and Monday (inclusive).
#### Values are set to 1 for these days and 0 otherwise.

In [ ]:
df["is_weekend"] = df["day_of_week"].isin(["Saturday", "Sunday", "Monday"]).astype(int)

In [ ]:
df

,event_id,ad_id,user_id,timestamp,day_of_week,time_of_day,event_type,campaign_id,ad_platform,ad_type,...,location,interests,year,month,month_name,day,hour,minute,second,is_weekend
0,65,1,e82c1,2025-06-28 10:56:52,Saturday,Morning,Impression,28,Facebook,Video,...,Joneshaven,art,2025,6,June,28,10,56,52,1
1,228,1,73d19,2025-07-11 22:34:14,Friday,Evening,Impression,28,Facebook,Video,...,Gallowaytown,"fashion, photography",2025,7,July,11,22,34,14,0
2,391,1,a7530,2025-05-21 08:41:59,Wednesday,Morning,Impression,28,Facebook,Video,...,Mariaview,"news, fashion, photography",2025,5,May,21,8,41,59,0
3,491,1,ca484,2025-06-26 14:32:05,Thursday,Afternoon,Impression,28,Facebook,Video,...,Blackburgh,"technology, lifestyle",2025,6,June,26,14,32,5,0
4,572,1,8c871,2025-05-19 05:44:09,Monday,Night,Impression,28,Facebook,Video,...,Nicolechester,fashion,2025,5,May,19,5,44,9,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399995,398935,200,9bef6,2025-05-28 16:57:07,Wednesday,Afternoon,Click,44,Instagram,Carousel,...,Jamesfurt,finance,2025,5,May,28,16,57,7,0
399996,399277,200,817c9,2025-08-05 22:00:50,Tuesday,Evening,Impression,44,Instagram,Carousel,...,Port Sara,fashion,2025,8,August,5,22,0,50,0
399997,399570,200,a5e53,2025-08-06 02:02:32,Wednesday,Night,Click,44,Instagram,Carousel,...,Marktown,travel,2025,8,August,6,2,2,32,0
399998,399762,200,a0f9a,2025-07-24 09:24:13,Thursday,Morning,Impression,44,Instagram,Carousel,...,South Brianberg,"fashion, food, fitness",2025,7,July,24,9,24,13,0


#### Save the prepared dataset as a CSV file named `df_after_prep.csv` without including index numbers.

In [ ]:
df.to_csv('df_after_prep.csv', index=False)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
df.to_csv('/content/drive/MyDrive/df_after_prep.csv', index=False)

Mounted at /content/drive
